# Taller Control 1
- Miguel Angel Alvarez Guzman   
- CC 1000871173


In [ ]:
! pip install ipywidgets numpy matplotlib scipy


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 39.0 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 890.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\Miguel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Punto 1
- Encontrar el modelo matemático para el siguiente sistema que representa el modelo de suspensión de 
una motocicleta. Asuma condiciones iniciales adecuadas. 

![Imagen Sistema 1](Sistema1.png)



## Solucion Punto 1

### Variables del Sistema
- $m_1$: masa 1
- $m_2$: masa 2
- $u(t)$: senoidal
- $x(t)$: desplazamiento de $m_1$
- $y(t)$: desplazamiento de $m_2$
- $k_1$: constante del resorte entre el seno y $m_1$
- $k_2$: constante del resorte entre $m_1$ y $m_2$
- $b$: constante de amortiguamiento viscosa entre $m_1$ y $m_2$

## Para la masa $m_1$:
### Fuerzas que actúan:

- Resorte $k_1$ respecto al Seno:
  
  $$
  F_{k1} = -k_1 (x - u)
  $$
  
- Resorte $k_2$ respecto a $m_2$:
  
  $$
  F_{k2} = -k_2 (x - y)
  $$

- Amortiguador $b$ respecto a $m_2$:
  
  $$
  F_b = --b (\dot{x} - \dot{y})
  $$

### Sumatoria de Fuerzas
$$
m_1 \ddot{x} = -k_1(x-u) - k_2(x-y) - b(\dot{x}-\dot{y})
$$


## Para la masa $m_2$:

### Fuerzas que actúan:

- Resorte $k_2$ respecto a $m_1$:
  
  $$
  F_{k2} = k_2 (x - y)
  $$

- Amortiguador $b$ respecto a $m_1$:
  
  $$
  F_b = b (\dot{x} - \dot{y})
  $$


### Sumatoria de Fuerzas

$$
m_2 \ddot{y} = k_2 (x - y) + b (\dot{x} - \dot{y})
$$


### Sistema resultante

$$
\boxed{
\begin{aligned}
m_1 \ddot{x} + (k_1 + k_2)x + b\dot{x} &= k_1 u(t) + k_2 y + b\dot{y} \\
m_2 \ddot{y} + k_2 y + b\dot{y} &= k_2 x + b\dot{x}
\end{aligned}
}
$$

### Condiciones inciales

$$
x(0) = 0, \quad y(0) = 0, \quad \dot{x}(0) = 0, \quad \dot{y}(0) = 0
$$

### Interpretacion $u(t)$
$$
u(t) = A \sin(\omega t)
$$

### Respuesta final

$$
\boxed{
\begin{aligned}
m_1 \ddot{x} + (k_1 + k_2)x + b\dot{x} &= k_1 A \sin(\omega t) + k_2 y + b\dot{y} \\
m_2 \ddot{y} + k_2 y + b\dot{y} &= k_2 x + b\dot{x}
\end{aligned}
}
$$

## Representación en Variables de Estado

Se Realiza cambio de variable

$$
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{bmatrix}
=
\begin{bmatrix}
x \\
\dot{x} \\
y \\
\dot{y}
\end{bmatrix}
$$


Las ecuaciones de estado son:

$$
\begin{aligned}
\dot{x}_1 &= x_2 \\
\dot{x}_2 &= \frac{1}{m_1} \left( k_1(u_e - x_1) + k_2(x_3 - x_1) + b(x_4 - x_2) \right) \\
\dot{x}_3 &= x_4 \\
\dot{x}_4 &= \frac{1}{m_2} \left( k_2(x_1 - x_3) + b(x_2 - x_4) \right)
\end{aligned}
$$

## Simulacion Punto 1



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider


m1 = 5.0     # kg
m2 = 15.0    # kg
k1 = 80.0   # N/m
k2 = 150.0  # N/m
b = 1000.0    # Ns/m


def entrada_senoidal(t, A=0.01, omega=5.0):
    return A * np.sin(omega * t)


def suspension(t, estado, m1, m2, k1, k2, b, A):
    x1, x2, x3, x4 = estado  # x, x', y, y'
    u = entrada_senoidal(t, A)
    
    dx1dt = x2
    dx2dt = (k1*(u - x1) + k2*(x3 - x1) + b*(x4 - x2)) / m1
    dx3dt = x4
    dx4dt = (k2*(x1 - x3) + b*(x2 - x4)) / m2
    
    return [dx1dt, dx2dt, dx3dt, dx4dt]

def simular_suspension(x0=0.0, dx0=0.0, y0=0.0, dy0=0.0, A=0):
    condiciones_iniciales = [x0, dx0, y0, dy0]
    
    t_span = (0, 50)  # 10 segundos
    t_eval = np.linspace(t_span[0], t_span[1], 1000)

    sol = solve_ivp(suspension, t_span, condiciones_iniciales, t_eval=t_eval, args=(m1, m2, k1, k2, b, A))

    plt.figure(figsize=(12,6))
    
    plt.plot(sol.t, sol.y[0], label='$x(t)$ (movimiento masa 1)')
    plt.plot(sol.t, sol.y[2], label='$y(t)$ (movimiento masa 2)')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Desplazamiento [m]')
    plt.title('Respuesta del sistema de suspensión a entrada senoidal')
    plt.grid(True)
    plt.legend()
    plt.show()

interact(simular_suspension,
         x0=FloatSlider(min=-0.05, max=0.05, step=0.005, value=0.0, description='x(0)'),
         dx0=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.0, description="x'(0)"),
         y0=FloatSlider(min=-0.05, max=0.05, step=0.005, value=0.0, description='y(0)'),
         dy0=FloatSlider(min=-1.0, max=1.0, step=0.1, value=0.0, description="y'(0)"),
         A=FloatSlider(min=0.0, max=0.1, step=0.01, value=0.01, description='A (amplitud)'),
        )

interactive(children=(FloatSlider(value=0.0, description='x(0)', max=0.05, min=-0.05, step=0.005), FloatSlider…

<function __main__.simular_suspension(x0=0.0, dx0=0.0, y0=0.0, dy0=0.0, A=0)>